# Feature Enginnering 6
***
这个 FE 我们将要将商户信息融入进去，试试效果。
我想了一个办法，对于 mer_merchant_category_id ，我应该创建一个特征is_diff_with_merchant_category_id， 将与merchant_category_id不同的项目标记为True。对于 mer_subsector_id，mer_state_id,city_id, category_1 同理，至于 category_2,直接onehot展开。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
usecol = [
    'category_1', 
    'category_2',
    'state_id', 
    'city_id',
    'subsector_id', 
    'merchant_id',
    'card_id',
    'merchant_category_id'
]

In [3]:
mer = pd.read_csv('/root/data/merchants.csv')
his = pd.read_csv('/root/data/historical_transactions.csv', usecols=usecol)
new = pd.read_csv('/root/data/new_merchant_transactions.csv', usecols=usecol)

In [4]:
print(mer.merchant_id.nunique(), mer.shape[0])

334633 334696


In [5]:
old_his_col = his.columns.tolist()
old_his_col.remove('card_id')

In [6]:
mer_count = mer.groupby('merchant_id').merchant_id.count()
mutiple_mer_id = mer_count[mer_count > 1].index

for mer_idx in mutiple_mer_id:
    index = mer[mer.merchant_id == mer_idx].index
    mer.drop(index[: -1], axis=0, inplace=True)
    
del mer_count, mutiple_mer_id, index, mer_idx

In [7]:
mer.columns = ['mer_'+i if i != 'merchant_id' else i for i in mer.columns]
his = pd.merge(his, mer, on='merchant_id', how='left')
new = pd.merge(new, mer, on='merchant_id', how='left')

In [8]:
#for i in his.columns:
#    if his[i].dtype != 'object':
#        his[i] = his[i].astype('float32')
#        
#for i in new.columns:
#    if new[i].dtype != 'object':
#        new[i] = new[i].astype('float32')

In [9]:
col = [
    'category_1', 
    'state_id', 
    'city_id',
    'subsector_id', 
    'merchant_category_id'
]


def search_diff(line1, line2):
    diffidx = [0 if line[0]==line[1] else 1 for line in zip(line1, line2)]
    return diffidx


for i in col:
    line1 = his[i]
    line2 = his['mer_'+i]
    his[f'is_diff_with_{i}'] = search_diff(line1, line2)
    his.drop([i], axis=1, inplace=True)
    his.drop(['mer_'+i], axis=1, inplace=True)
    del line1, line2


for i in col:
    line1 = new[i]
    line2 = new['mer_'+i]
    new[f'is_diff_with_{i}'] = search_diff(line1, line2)
    new.drop([i], axis=1, inplace=True)
    new.drop(['mer_'+i], axis=1, inplace=True)
    del line1, line2

In [10]:
dum_col = [
    'mer_category_2',
    'mer_category_4',
    'mer_most_recent_sales_range',
    'mer_most_recent_purchases_range'
]
his = pd.get_dummies(his, columns=dum_col, dummy_na=True, dtype='bool')
new = pd.get_dummies(new, columns=dum_col, dummy_na=True, dtype='bool')

In [11]:
his.drop(['merchant_id', 'category_2'], axis=1, inplace=True)
new.drop(['merchant_id', 'category_2'], axis=1, inplace=True)

In [12]:
his.fillna(0, inplace=True)
new.fillna(0, inplace=True)

his.replace(np.inf, 0, inplace=True)
new.replace(np.inf, 0, inplace=True)

In [13]:
def cal_mode(serise):
    return serise.mode()[0]


def agg_dataframe(df):
    agg = {'mer_category_2_1.0':['mean'],
           'mer_category_2_2.0':['mean'], 
           'mer_category_2_3.0':['mean'],
           'mer_category_2_4.0':['mean'], 
           'mer_category_2_5.0':['mean'], 
           'mer_category_2_nan':['mean'],
           'mer_category_4_N':['mean'],
           'mer_category_4_Y':['mean'],
           'mer_category_4_nan':['mean'],
           'mer_most_recent_sales_range_A':['mean'],
           'mer_most_recent_sales_range_B':['mean'],
           'mer_most_recent_sales_range_C':['mean'],
           'mer_most_recent_sales_range_D':['mean'],
           'mer_most_recent_sales_range_E':['mean'],
           'mer_most_recent_sales_range_nan':['mean'],
           'mer_most_recent_purchases_range_A':['mean'],
           'mer_most_recent_purchases_range_B':['mean'],
           'mer_most_recent_purchases_range_C':['mean'],
           'mer_most_recent_purchases_range_D':['mean'],
           'mer_most_recent_purchases_range_E':['mean'], 
           'mer_most_recent_purchases_range_nan':['mean'],
           'is_diff_with_category_1':['mean'],
           'is_diff_with_state_id':['mean'],
           'is_diff_with_city_id':['mean'],
           'is_diff_with_subsector_id':['mean'], 
           'is_diff_with_merchant_category_id':['mean'],
           'mer_numerical_1':['min', 'max', 'std', 'mean', 'sum'],
           'mer_numerical_2':['min', 'max', 'std', 'mean', 'sum'],
           'mer_avg_sales_lag3':['min', 'max', 'std', 'mean', 'sum'],
           'mer_avg_purchases_lag3':['min', 'max', 'std', 'mean', 'sum'],
           'mer_active_months_lag3':['min', 'max', 'std', 'mean', 'sum'],
           'mer_avg_sales_lag6':['min', 'max', 'std', 'mean', 'sum'],
           'mer_avg_purchases_lag6':['min', 'max', 'std', 'mean', 'sum'],
           'mer_active_months_lag6':['min', 'max', 'std', 'mean', 'sum'],
           'mer_avg_sales_lag12':['min', 'max', 'std', 'mean', 'sum'],
           'mer_avg_purchases_lag12':['min', 'max', 'std', 'mean', 'sum'],
           'mer_active_months_lag12':['min', 'max', 'std', 'mean', 'sum'],
           'mer_merchant_group_id':[cal_mode]}
    
    df_agg = df.groupby('card_id').agg(agg)
    df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]
    df_agg.reset_index(inplace=True)
    return df_agg

In [14]:
agg_his = agg_dataframe(his)
agg_his.columns = ['his_' + c if c != 'card_id' else c for c in agg_his.columns]
agg_new = agg_dataframe(new)
agg_new.columns = ['new_' + c if c != 'card_id' else c for c in agg_new.columns]

In [15]:
train = pd.read_csv('/root/tempfile/train_final121201.csv')
test = pd.read_csv('/root/tempfile/test_final121201.csv')

In [18]:
train = pd.merge(train, agg_his, on='card_id', how='left')
train = pd.merge(train, agg_new, on='card_id', how='left')

test = pd.merge(test, agg_his, on='card_id', how='left')
test = pd.merge(test, agg_new, on='card_id', how='left')

In [19]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [23]:
train.to_csv('/root/tempfile/train_final122812.csv',index=False)
test.to_csv('/root/tempfile/test_final122812.csv',index=False)